In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
# Any results you write to the current directory are saved as output.

In [ ]:
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Activation,Dropout
from keras.layers.normalization import BatchNormalization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
#traindata.h5 contains the entire data,only the name is incorrect
with h5py.File('/kaggle/input/myass2/traindata.h5', 'r') as file:
    ids = pd.DataFrame(np.array(np.squeeze(file['ids'])),columns=['id'])
    full_data = np.array(file['images']).reshape(-1,28,28,3)
labels = pd.read_csv('/kaggle/input/datalabel/datafile1.csv')
#print(labels.shape)
Y = labels['label']
Y = pd.get_dummies(Y)
#split the entire data into train and test
X_train,X_test,Y_train,Y_test = train_test_split(full_data,Y,random_state=32,test_size=0.2)

model = Sequential()
model.add(Conv2D(32,kernel_size=2,kernel_initializer='he_normal',input_shape=(28,28,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64,kernel_size=3,kernel_initializer='he_normal',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,kernel_size=2,kernel_initializer='he_normal',padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,kernel_size=2,kernel_initializer='he_normal',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(96,kernel_initializer='glorot_uniform',activation='softmax'))

#Nadam is also an optimizer
model.compile(loss='categorical_crossentropy',optimizer='Nadam',metrics=["accuracy"])
model.summary()

            

In [ ]:
epochs=5
datagen = ImageDataGenerator(rotation_range=0,width_shift_range=0,height_shift_range=0,horizontal_flip=False,zoom_range=0)
train_batch = datagen.flow(X_train,Y_train,batch_size=500)
model.fit_generator(train_batch,steps_per_epoch=X_train.shape[0]//32,epochs=epochs)


In [ ]:
#print(X_train)
#check if the actual test labels are predicted correctly
testloss,testacc =  model.evaluate(X_test,Y_test,verbose=1)
print('testacc',testacc)


In [ ]:
#check prediction by plotting
X_test1 = np.expand_dims(X_test[0], axis=0)
print(X_test1.shape)
pred =  model.predict(X_test1)
print(np.argmax(pred))
#for i in range(1):  
#plt.subplot(4, 10, i+1)
plt.imshow(X_test1.reshape(28,28,3),cmap=plt.cm.hsv)
plt.title(f"{Y_test.iat[0,37]}")
plt.axis('off')
plt.subplots_adjust(wspace=0.3, hspace=0.3)
plt.show()